In [502]:
import yfinance as yf
import pandas as pd
import math
import numpy_financial as npf
import statistics
import datetime
from dateutil.relativedelta import relativedelta
from pandas.core.frame import DataFrame
from scipy.stats import gmean


## 設定定期投入金額起始結束日期投資組合配比

In [528]:
start='2020-1-1'
end='2022-12-31'
ratio=[0.5,0.25,0.25]
stock=['AAPL','TSLA','AMZN']
cashflows=['None','Contribute_fixed']
contribution_amount=10000
contribution_amount_type=['fixed', 'drift']

In [504]:
PMT=contribution_amount
start_dt=datetime.datetime.strptime(start,"%Y-%m-%d")
start_dt=(start_dt-relativedelta(months=+1)).strftime("%Y-%m-%d")
stock_name=[]
stock_list=[]
end_dt=datetime.datetime.strptime(end,"%Y-%m-%d")
end_dt=(end_dt+datetime.timedelta(days=1)).strftime("%Y-%m-%d")
for i in range(len(stock)):
#     a=round(yf.download(stock[i],start=start_dt,end=end_dt),2)
    a=yf.download(stock[i],start=start_dt,end=end_dt)
    stock_list.append(a)

# print(stock_list[0])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [505]:
for i in stock_list:
    i.drop(["Open"],axis=1,inplace=True)
    i.drop(["High"],axis=1,inplace=True)
    i.drop(["Low"],axis=1,inplace=True)
    i.drop(["Close"],axis=1,inplace=True)
    i.drop(["Volume"],axis=1,inplace=True)

In [506]:

if len(stock)==1:
    data=stock_list[0].copy()
    data.rename(columns={'Adj Close':stock[0]},inplace=True)
else:
    data=stock_list[0].copy()
    data.rename(columns={'Adj Close':stock[0]},inplace=True)
    for i in range(1,len(stock)):
        data=pd.merge(data,stock_list[i],left_index=True,right_index=True,how='inner')
        data.rename(columns={'Adj Close':stock[i]},inplace=True)
# data.tail(1)

In [536]:
df=data.copy()
dfm=df.resample('BM').last()
dfm=dfm.loc[start_dt:end]
# dfm=round(dfm,2)
# dfm.to_excel('20230329_1.xlsx')
# dfm

In [508]:
# print(PMT*ratio[0]/(dfm[stock[0]].iloc[0]))

# unit1=[]
# unit2=[]
# unit3=[]
# unit=[unit1,unit2,unit3]
# fl=[0]
# unit1.append(PMT*ratio[0]/(dfm[stock[0]].iloc[0]))
# unit2.append(PMT*ratio[1]/(dfm[stock[1]].iloc[0]))
# unit3.append(PMT*ratio[2]/(dfm[stock[2]].iloc[0]))
# fl.append(unit1[0]*(dfm[stock[0]].iloc[1])+unit2[0]*(dfm[stock[1]].iloc[1])+unit3[0]*(dfm[stock[2]].iloc[1]))
# print(unit1,'\\',unit2,'\\',unit3,'\\',fl)
# unit1.append((PMT*(unit1[0]*dfm[stock[0]].iloc[1]/fl[1]))/(dfm[stock[0]].iloc[1])+unit1[0])
# unit2.append((PMT*(unit2[0]*dfm[stock[1]].iloc[1]/fl[1]))/(dfm[stock[1]].iloc[1])+unit2[0])
# unit3.append((PMT*(unit3[0]*dfm[stock[2]].iloc[1]/fl[1]))/(dfm[stock[2]].iloc[1])+unit3[0])
# fl.append(unit1[1]*(dfm[stock[0]].iloc[2])+unit2[1]*(dfm[stock[1]].iloc[2])+unit3[1]*(dfm[stock[2]].iloc[2]))
# print((PMT*(unit1[0]*dfm[stock[0]].iloc[1]/fl[1]))/(dfm[stock[0]].iloc[1]))
# print(unit1,'\\',unit2,'\\',unit3,'\\',fl)
# print(unit[0],unit[1],unit[2])
# for i in range(len(dfm)):
#     for j in range(len(stock)):
#         unit[j].append((PMT*(unit[j][i]*dfm[stock[j]].iloc[i+1]/fl[i+1]))/(dfm[stock[j]].iloc[i+1])+unit[j][i])
#         fl.append(unit[j][i+1]*(dfm[stock[j]].iloc[2])+unit2[1]*(dfm[stock[1]].iloc[2])+unit3[1]*(dfm[stock[2]].iloc[2]))
# fl.append(unit1[1]*(dfm[stock[0]].iloc[2])+unit2[1]*(dfm[stock[1]].iloc[2])+unit3[1]*(dfm[stock[2]].iloc[2]))

In [509]:

TotalCost_list=[]
for i in range(1,len(dfm)+1):
    TotalCost=PMT*i
    TotalCost_list.append(TotalCost)
dfm['total_cost']=TotalCost_list

In [510]:
# 各支股票股數計算
unit_name_list=[]
# preNAV_name_list=[]
# for i in range(len(stock)):
#     unit_name_list.append("unit"+str(k+1))
#     preNAV_name_list.append("preNAV"+str(k+1))
    
for k in range(len(dfm.columns[0:len(stock_list)])):
    preNAV_list=[0]
    unit_list=[PMT*ratio[k]/dfm[dfm.columns[k]].iloc[0]]
    unit_name_list.append("unit"+str(k+1))
    for i in dfm[dfm.columns[k]].iloc[1:]:
        unit=unit_list[-1]+PMT*ratio[k]/i
        unit=unit
        unit_list.append(unit)
    dfm[unit_name_list[k]]=unit_list

dfm.head()

,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3
Date,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620
2020-01-31,71.639000,43.371334,100.435997,20000,144.455241,147.283910,51.950094
2020-02-28,66.962502,44.532665,94.187500,30000,219.123898,203.422476,78.492894
2020-03-31,58.097500,34.933334,97.486000,40000,305.186121,274.987360,104.137602
2020-04-30,67.334999,52.125332,123.699997,50000,379.441710,322.948685,124.347789


In [511]:
# 各支股票淨值計算
preNAV_name_list=[]
for k in range(len(dfm.columns[0:len(stock_list)])):
    preNAV_list=[0]
    preNAV_name_list.append("preNAV"+str(k+1))
    for i,j in enumerate(dfm[dfm.columns[k]].iloc[1:]):
        preNAV=dfm[unit_name_list[k]].iloc[i]*j
#         preNAV=dfm[bb[k]].iloc[i]*j
        preNAV_list.append(preNAV)
    dfm[preNAV_name_list[k]]=preNAV_list

dfm.head()

,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3
Date,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000
2020-01-31,71.639000,43.371334,100.435997,20000,144.455241,147.283910,51.950094,5348.629034,3887.899643,2717.659507
2020-02-28,66.962502,44.532665,94.187500,30000,219.123898,203.422476,78.492894,9673.084327,6558.945039,4893.049499
2020-03-31,58.097500,34.933334,97.486000,40000,305.186121,274.987360,104.137602,12730.550607,7106.225377,7651.958316
2020-04-30,67.334999,52.125332,123.699997,50000,379.441710,322.948685,124.347789,20549.707165,14333.807384,12881.821102


In [512]:
# return_name_list=[]
# cc=[]
# for k in range(len(dfm.columns[0:len(stock_list)])):
#     return_name_list.append("return"+str(k+1)+'(%)')
# #     dfm[cc[k]]=(dfm[aa[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100
#     dfm[return_name_list[k]]=round((dfm[preNAV_name_list[k]]/(dfm['total_cost']*ratio[k]-PMT*ratio[k])-1)*100,2)

# dfm.head()

In [513]:

dfm["final"]=dfm[preNAV_name_list[0]].copy()
for i in range(1,len(stock)):
    dfm["final"]+=dfm[preNAV_name_list[i]]
for i in range(len(dfm)):
    dfm["final"][i]+=PMT
dfm

C:\Users\PHIL\AppData\Local\Temp\ipykernel_22248\3944490220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm["final"][i]+=PMT


,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3,final
Date,,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000,10000.000000
2020-01-31,71.639000,43.371334,100.435997,20000,144.455241,147.283910,51.950094,5348.629034,3887.899643,2717.659507,21954.188184
2020-02-28,66.962502,44.532665,94.187500,30000,219.123898,203.422476,78.492894,9673.084327,6558.945039,4893.049499,31125.078865
2020-03-31,58.097500,34.933334,97.486000,40000,305.186121,274.987360,104.137602,12730.550607,7106.225377,7651.958316,37488.734300
2020-04-30,67.334999,52.125332,123.699997,50000,379.441710,322.948685,124.347789,20549.707165,14333.807384,12881.821102,57765.335651
2020-05-29,71.676003,55.666668,122.118500,60000,449.200064,367.858864,144.819708,27196.864923,17977.477208,15185.165423,70359.507553
2020-06-30,70.902496,71.987335,137.940994,70000,519.719443,402.587195,162.943399,31849.405904,26481.179322,19976.574450,88307.159676
2020-07-31,74.397499,95.384003,158.233994,80000,586.926003,428.797040,178.742785,38665.826785,38400.378044,25783.184724,112849.389553
2020-08-31,81.476501,166.106674,172.548004,90000,648.293390,443.847610,193.231507,47820.677359,71226.050269,30841.710850,159888.438478


In [514]:
# monthly_return
stdev=dfm.copy()
monthly_return=[(stdev["final"][0]/PMT)-1]
for i in range(1,len(stdev)):
    mreturn=((stdev["final"][i]-PMT)/stdev["final"][i-1])-1
    monthly_return.append(mreturn*100)

dfm["monthly return(%)"]=monthly_return
dfm["monthly return(%)"]=round(dfm["monthly return(%)"],2)
# dfm

In [515]:
def MDD(dr):
#     dr=df.pct_change(1)
    r=dr.add(1).cumprod()
    dd=r.div(r.cummax()).sub(1)
    mdd=dd.min()
    end=dd.idxmin()
    start=r.loc[:end[0]].idxmax()
    days=end-start
    return mdd[0], start[0], end[0], days[0]

# MAX Drawdown


In [516]:
ddddd=(dfm["monthly return(%)"][0:]/100).copy()
ddddd=ddddd.to_frame()
MDD(ddddd)
# print(ddddd)

(-0.5302453747705855,
 Timestamp('2021-11-30 00:00:00'),
 Timestamp('2022-12-30 00:00:00'),
 Timedelta('395 days 00:00:00'))

# IRR


In [517]:
# T_R=TR/100
# print('總回報',TR,'%')
# months=len(dfm)
# print("共有:",months,'月')
# IRR=(1+T_R)**(12/months)-1
# print('內部報酬率(IRR):',round(IRR*100,2),'%')

# stdev

In [518]:
df_stdev=dfm.copy()
# df_stdev=df_stdev["monthly return(%)"]
risk=df_stdev["monthly return(%)"][1:].std()
risk_Y=risk*12**0.5
risk_Y=round(risk_Y,2)
ar=df_stdev["monthly return(%)"][1:].mean()*12
std=risk_Y
sp=ar/std
sp=round(sp,2)
# sortino
sortino=[]
for i in range(1,len(df_stdev)):
    if df_stdev["monthly return(%)"][i]<0:
        sortino.append(df_stdev["monthly return(%)"][i])
    else:
        sortino.append(0)
# print(sortino,len(sortino),DataFrame(sortino).std())
sttd_D=DataFrame(sortino).std()*12**0.5
stt=ar/sttd_D
stt=round(stt,3)
# df_stdev["monthly return(%)"][1:]
# std_D=df_stdev["monthly return(%)"][1:][df_stdev["monthly return(%)"]<0].std()*12**0.5
# st=ar/std_D
# st=round(st,3)


print("年化標準差:",risk_Y,"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',stt[0])


# df_stdev.plot()

年化標準差: 44.99 %
Sharpe ratio: 0.71
sortino ratio: 1.684


In [519]:
dfm_c=dfm.copy()
dfm_c['final']=round(dfm_c['final'])

dfm_c

,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3,final,monthly return(%)
Date,,,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000,10000.0,0.00
2020-01-31,71.639000,43.371334,100.435997,20000,144.455241,147.283910,51.950094,5348.629034,3887.899643,2717.659507,21954.0,19.54
2020-02-28,66.962502,44.532665,94.187500,30000,219.123898,203.422476,78.492894,9673.084327,6558.945039,4893.049499,31125.0,-3.78
2020-03-31,58.097500,34.933334,97.486000,40000,305.186121,274.987360,104.137602,12730.550607,7106.225377,7651.958316,37489.0,-11.68
2020-04-30,67.334999,52.125332,123.699997,50000,379.441710,322.948685,124.347789,20549.707165,14333.807384,12881.821102,57765.0,27.41
2020-05-29,71.676003,55.666668,122.118500,60000,449.200064,367.858864,144.819708,27196.864923,17977.477208,15185.165423,70360.0,4.49
2020-06-30,70.902496,71.987335,137.940994,70000,519.719443,402.587195,162.943399,31849.405904,26481.179322,19976.574450,88307.0,11.30
2020-07-31,74.397499,95.384003,158.233994,80000,586.926003,428.797040,178.742785,38665.826785,38400.378044,25783.184724,112849.0,16.47
2020-08-31,81.476501,166.106674,172.548004,90000,648.293390,443.847610,193.231507,47820.677359,71226.050269,30841.710850,159888.0,32.82


In [520]:
dffff=data.resample('BM').last().copy()
# dffff=dffff["0050"]*0.5+dffff["0050"]+dffff["0050"]*0.2
# print('內部報酬率(IRR):',round(IRR*100,2),'%')
# print("標準差(stdev):",round(risk*100,2),"%")
# print(dfm)
CAGR=round(((dfm['final'][-1]/dfm['total_cost'][0])**(1/((len(dfm)-1)/12))-1)*100,2)
print("========================================================")

# print('分別占比為(%):',ratio[0]*100,',',ratio[1]*100,',',ratio[2]*100)
# print("回測區間:",start,'~',end)
# print('總報酬(Total Return):',round((dfm['return1(%)'][-1]*0.5+dfm['return2(%)'][-1]*0.25+dfm['return3(%)'][-1]*0.25),2),'%')

print("標的:",stock)
print("回測區間:",start,'~',end)
print("總損益:",dfm_c['final'][-1])
print("CAGR:",CAGR,'%')
print("標準差:","{:.2f}".format(risk_Y),"%")
print('Sharpe ratio:',sp)
print('sortino ratio:',stt[0])
print('最大回撤(MAX Drawdown):',round(MDD(ddddd)[0]*100,2),'%')
# print('最大回撤區間:',MDD(ddddd)[1].strftime('%Y-%m-%d'),'~',MDD(ddddd)[2].strftime('%Y-%m-%d'))
print("========================================================")


標的: ['GOOGL', 'TSLA', 'AMZN']
回測區間: 2020-1-1 ~ 2022-12-31
總損益: 319938.0
CAGR: 217.46 %
標準差: 44.99 %
Sharpe ratio: 0.71
sortino ratio: 1.684
最大回撤(MAX Drawdown): -53.02 %


In [521]:
dfm_c


,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3,final,monthly return(%)
Date,,,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000,10000.0,0.00
2020-01-31,71.639000,43.371334,100.435997,20000,144.455241,147.283910,51.950094,5348.629034,3887.899643,2717.659507,21954.0,19.54
2020-02-28,66.962502,44.532665,94.187500,30000,219.123898,203.422476,78.492894,9673.084327,6558.945039,4893.049499,31125.0,-3.78
2020-03-31,58.097500,34.933334,97.486000,40000,305.186121,274.987360,104.137602,12730.550607,7106.225377,7651.958316,37489.0,-11.68
2020-04-30,67.334999,52.125332,123.699997,50000,379.441710,322.948685,124.347789,20549.707165,14333.807384,12881.821102,57765.0,27.41
2020-05-29,71.676003,55.666668,122.118500,60000,449.200064,367.858864,144.819708,27196.864923,17977.477208,15185.165423,70360.0,4.49
2020-06-30,70.902496,71.987335,137.940994,70000,519.719443,402.587195,162.943399,31849.405904,26481.179322,19976.574450,88307.0,11.30
2020-07-31,74.397499,95.384003,158.233994,80000,586.926003,428.797040,178.742785,38665.826785,38400.378044,25783.184724,112849.0,16.47
2020-08-31,81.476501,166.106674,172.548004,90000,648.293390,443.847610,193.231507,47820.677359,71226.050269,30841.710850,159888.0,32.82


In [522]:
dffff=dfm.copy()
# unit_name_list=[]
# preNAV_name_list=[]
# print(dffff['final'])
final_l=dffff['final'].copy()
for i in range(len(dffff)):
    final_l[i]=PMT


for i in range(1,len(dffff)):
    for j in range(len(stock)):
        dffff[preNAV_name_list[j]].iloc[i]=dffff[unit_name_list[j]].iloc[i-1]*dffff[stock[j]].iloc[i]
        final_l[i]+=dffff[preNAV_name_list[j]].iloc[i]
    dffff['final'][i]=final_l[i]    
    for j in range(len(stock)):        
        dffff[unit_name_list[j]].iloc[i]=(((dffff[preNAV_name_list[j]].iloc[i]/(dffff['final'].iloc[i]-PMT))*PMT)/dffff[stock[j]].iloc[i])
        dffff[unit_name_list[j]].iloc[i]+=dffff[unit_name_list[j]].iloc[i-1]
     
        
#         print(dffff['final'][i],dffff[preNAV_name_list[j]].iloc[i])
#         print()
#         print("======")
print(dffff['final'].iloc[0])
dffff

C:\Users\PHIL\AppData\Local\Temp\ipykernel_22248\4281662975.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffff[preNAV_name_list[j]].iloc[i]=dffff[unit_name_list[j]].iloc[i-1]*dffff[stock[j]].iloc[i]
C:\Users\PHIL\AppData\Local\Temp\ipykernel_22248\4281662975.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffff['final'][i]=final_l[i]
C:\Users\PHIL\AppData\Local\Temp\ipykernel_22248\4281662975.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

10000.0


,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3,final,monthly return(%)
Date,,,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000,10000.000000,0.00
2020-01-31,71.639000,43.371334,100.435997,20000,137.116670,164.630220,49.693884,5348.629034,3887.899643,2717.659507,21954.188184,19.54
2020-02-28,66.962502,44.532665,94.187500,30000,201.813752,242.309287,73.141429,9181.675217,7331.422492,4680.542707,31193.640417,-3.78
2020-03-31,58.097500,34.933334,97.486000,40000,275.684597,331.002904,99.913733,11724.874439,8464.671333,7130.265376,37319.811148,-11.68
2020-04-30,67.334999,52.125332,123.699997,50000,332.908843,399.709650,120.652970,18563.222071,17253.636218,12359.328512,58176.186802,27.41
2020-05-29,71.676003,55.666668,122.118500,60000,387.622157,465.401625,140.482193,23861.575052,22250.504362,14733.959676,70846.039090,4.49
2020-06-30,70.902496,71.987335,137.940994,70000,435.855072,523.312859,157.962788,27483.378539,33503.022815,19378.253413,90364.654768,11.30
2020-07-31,74.397499,95.384003,158.233994,80000,476.461192,572.066920,172.679276,32426.527294,49915.675103,24995.082716,117337.285113,16.47
2020-08-31,81.476501,166.106674,172.548004,90000,505.577620,607.025791,183.231664,38820.391007,95024.133480,29795.464381,173639.988868,32.82


In [523]:
# monthly_return
stdev_2=dffff.copy()
monthly_return=[(stdev_2["final"][0]/PMT)-1]
for i in range(1,len(stdev_2)):
    mreturn=((stdev_2["final"][i]-PMT)/stdev_2["final"][i-1])-1
    monthly_return.append(mreturn*100)

dffff["monthly return(%)"]=monthly_return
# dffff["monthly return(%)"]=round(dffff["monthly return(%)"],2)
dffff

,GOOGL,TSLA,AMZN,total_cost,unit1,unit2,unit3,preNAV1,preNAV2,preNAV3,final,monthly return(%)
Date,,,,,,,,,,,,
2019-12-31,66.969498,27.888666,92.391998,10000,74.660856,89.642150,27.058620,0.000000,0.000000,0.000000,10000.000000,0.000000
2020-01-31,71.639000,43.371334,100.435997,20000,137.116670,164.630220,49.693884,5348.629034,3887.899643,2717.659507,21954.188184,19.541882
2020-02-28,66.962502,44.532665,94.187500,30000,201.813752,242.309287,73.141429,9181.675217,7331.422492,4680.542707,31193.640417,-3.464249
2020-03-31,58.097500,34.933334,97.486000,40000,275.684597,331.002904,99.913733,11724.874439,8464.671333,7130.265376,37319.811148,-12.418651
2020-04-30,67.334999,52.125332,123.699997,50000,332.908843,399.709650,120.652970,18563.222071,17253.636218,12359.328512,58176.186802,29.090114
2020-05-29,71.676003,55.666668,122.118500,60000,387.622157,465.401625,140.482193,23861.575052,22250.504362,14733.959676,70846.039090,4.589253
2020-06-30,70.902496,71.987335,137.940994,70000,435.855072,523.312859,157.962788,27483.378539,33503.022815,19378.253413,90364.654768,13.435636
2020-07-31,74.397499,95.384003,158.233994,80000,476.461192,572.066920,172.679276,32426.527294,49915.675103,24995.082716,117337.285113,18.782377
2020-08-31,81.476501,166.106674,172.548004,90000,505.577620,607.025791,183.231664,38820.391007,95024.133480,29795.464381,173639.988868,39.461203


In [524]:
df_stdev_2=dffff.copy()
# df_stdev=df_stdev["monthly return(%)"]
risk_2=df_stdev_2["monthly return(%)"][1:].std()
risk_Y_2=risk_2*12**0.5
risk_Y_2=risk_Y_2
ar_2=df_stdev_2["monthly return(%)"][1:].mean()*12
std_2=risk_Y_2
sp_2=ar_2/std_2
sp_2=round(sp_2,2)
# sortino
sortino_2=[]
for i in range(1,len(df_stdev_2)):
    if df_stdev_2["monthly return(%)"][i]<0:
        sortino_2.append(df_stdev_2["monthly return(%)"][i])
    else:
        sortino_2.append(0)
# print(sortino,len(sortino),DataFrame(sortino).std())
sttd_D_2=DataFrame(sortino_2).std()*12**0.5
stt_2=ar_2/sttd_D_2
stt_2=round(stt_2,3)
df_stdev_2["monthly return(%)"][1:]
std_D_2=df_stdev_2["monthly return(%)"][1:][df_stdev_2["monthly return(%)"]<0].std()*12**0.5
st_2=ar_2/std_D_2
st_2=round(st_2,3)


print("年化標準差:",risk_Y_2,"%")
print('Sharpe ratio:',sp_2)
print('sortino ratio:',stt_2[0])
print('sortino ratio:',st_2)


年化標準差: 53.450942151286064 %
Sharpe ratio: 0.68
sortino ratio: 1.627
sortino ratio: 1.665


In [525]:
ddddd_2=(dffff["monthly return(%)"][0:]/100).copy()
ddddd_2=ddddd_2.to_frame()
MDD(ddddd_2)

(-0.598299197889353,
 Timestamp('2021-11-30 00:00:00'),
 Timestamp('2022-12-30 00:00:00'),
 Timedelta('395 days 00:00:00'))

In [527]:
# print(dfm)
CAGR_2=round(((dffff['final'][-1]/dffff['total_cost'][0])**(1/((len(dffff)-1)/12))-1)*100,2)
print("========================================================")

# print('分別占比為(%):',ratio[0]*100,',',ratio[1]*100,',',ratio[2]*100)
# print("回測區間:",start,'~',end)
# print('總報酬(Total Return):',round((dfm['return1(%)'][-1]*0.5+dfm['return2(%)'][-1]*0.25+dfm['return3(%)'][-1]*0.25),2),'%')

print("標的:",stock)
print("回測區間:",start,'~',end)
print("總損益:",round(dffff['final'][-1]))
print("CAGR:",CAGR_2,'%')
print("標準差:","{:.2f}".format(risk_Y_2),"%")
print('Sharpe ratio:',sp_2)
print('sortino ratio:',stt_2[0])
print('最大回撤(MAX Drawdown):',round(MDD(ddddd_2)[0]*100,2),'%')
print('最大回撤區間:',MDD(ddddd_2)[1].strftime('%Y-%m-%d'),'~',MDD(ddddd_2)[2].strftime('%Y-%m-%d'))
print("========================================================")


標的: ['GOOGL', 'TSLA', 'AMZN']
回測區間: 2020-1-1 ~ 2022-12-31
總損益: 302341
CAGR: 211.53 %
標準差: 53.45 %
Sharpe ratio: 0.68
sortino ratio: 1.627
最大回撤(MAX Drawdown): -59.83 %
最大回撤區間: 2021-11-30 ~ 2022-12-30


In [542]:
dffff_c=dffff.copy()
# unit_name_list=[]
# preNAV_name_list=[]
for i in range(len(stock)):
    dffff_c[stock[i]]=round(dffff_c[stock[i]],2)
    dffff_c[unit_name_list[i]]=round(dffff_c[stock[i]])
    dffff_c[preNAV_name_list[i]]=round(dffff_c[preNAV_name_list[i]])
    dffff_c['final']=round(dffff_c['final'])
    dffff_c['monthly return(%)']=round(dffff_c['monthly return(%)'],2)

